In [157]:
%reload_ext autoreload
%autoreload 2

In [158]:
# Modulos da propria aplicacao
from limpeza import limpeza_posts, limpeza_texto
from repository import nerds_viajantes, wikipedia, mongo_utils
from treinamento import dicionario, treinamento_lda
from treinamento.treinamento_lda import TreinamentoLda

from repository.mongo_utils import get_pages_content_collection
from repository.wikipedia import WikipediaRepo

wikipedia_repo = WikipediaRepo(collection=get_pages_content_collection())

In [159]:
from similarity.similarity import SimilarityCalculator

In [160]:
# Modelos de pacotes
import pandas as pd
import numpy as np

## Trabalho com base de treinamento

In [161]:
# Carregar posts
published = nerds_viajantes.read_published()

# Remove posts desnecessarios
posts = limpeza_posts.limpar_posts(published)

# Limpa texto e gera documentos para treinamento
documentos = posts['content'].apply(limpeza_texto.limpar_texto)

### Ajuste de modelo para posts do Nerds Viajantes

In [162]:
# Ajusta modelo LDA para os posts do nerds viajantes. O resultado gerado contem o dicionario, o corpus de dados e o modelo gerado
treinamento_lda = TreinamentoLda(num_topics=100, passes=2)
lda_nerds_viajantes = treinamento_lda.ajustar_modelo(documentos)

Ajustando modelo com 100 topicos e 2 passes


### Definicao de probabilidades de topicos para posts do Nerds Viajantes

In [163]:
probabilidades_topicos_nerds_viajantes = [treinamento_lda.calcular_probabilidades_documento(dnv, lda_nerds_viajantes) for dnv in documentos]
np.array(probabilidades_topicos_nerds_viajantes).shape

(378, 100)

### Teste com classe

## Trabalho com base da Wikipedia

In [164]:
pages = wikipedia_repo.find_all()
pages_df = pd.DataFrame(pages)
pages_df = pages_df.drop(['_id', 'ns', 'type', 'download', 'categories', 'wikitext', 'text'], axis='columns')

In [165]:
# Limpa texto e gera documentos para treinamento
documentos_wikipedia = pages_df['text_clean'].apply(limpeza_texto.limpar_texto)

### Definicao de topicos para documentos da Wikipedia

- Quantidade de documentos: 2252
- Indice do maior documento: 123
- Tamanho do maior documento: 10018 tokens
- Tempo gasto para definir tópicos de maior documento: 0.02167201042175293 segundos
- Tempo estimado para todos os documentos (com base no tempo do maior): 18.83 segundos
    * Na verdade este valor variou muito, não é fixo, mas a maioria das vezes ficou em torno de 20 segundos

In [166]:
import time

def avaliar_tempos_definicao_topicos_wikipedia():
    print(f'Quantidade de documentos: {len(documentos_wikipedia)}')

    tamanhos = [len(d) for d in documentos_wikipedia]
    indice_maior_documento = np.argmax(tamanhos)
    print(f'Indice do maior documento: {indice_maior_documento}')
    print(f'Tamanho do maior documento: {tamanhos[indice_maior_documento]} tokens')

    maior_documento = documentos_wikipedia[indice_maior_documento]
    start_time = time.time()
    treinamento_lda.calcular_probabilidades_documento(maior_documento, lda_nerds_viajantes)
    tempo_gasto_maior_documento = time.time() - start_time
    print(f"Tempo gasto na definicao dos topicos do maior documento: {tempo_gasto_maior_documento:.2f} segundos")
    print(f'Tempo estimado para todos documentos: {len(documentos_wikipedia) * tempo_gasto_maior_documento:.2f} segundos')

# avaliar_tempos_definicao_topicos_wikipedia()

### Calculo de probabilidade de topicos para paginas da wikipedia

In [167]:
start_time = time.time()
probabilidades_topicos_wikipedia = [
    treinamento_lda.calcular_probabilidades_documento(dw, lda_nerds_viajantes) for dw in documentos_wikipedia]
elapsed_time = time.time() - start_time
print(f'Tempo total gasto na definicao dos topicos: {elapsed_time:.2f} segundos')

Tempo total gasto na definicao dos topicos: 7.56 segundos


## Calculo de semelhanca entre posts do Nerds Viajantes e Posts da Wikipedia

In [168]:
print(f'Quantidade de posts do nerds viajantes: {len(probabilidades_topicos_nerds_viajantes)}')
print(f'Tamanho do array de probabilidades de posts do nerds viajantes: {len(probabilidades_topicos_nerds_viajantes[0])}')
print(f'Quantidade de paginas da wikipedia: {len(probabilidades_topicos_wikipedia)}')
print(f'Tamanho do array de probabilidades de paginas da wikipedia: {len(probabilidades_topicos_wikipedia[0])}')

Quantidade de posts do nerds viajantes: 378
Tamanho do array de probabilidades de posts do nerds viajantes: 100
Quantidade de paginas da wikipedia: 2252
Tamanho do array de probabilidades de paginas da wikipedia: 100


### Problema encontrado

Ao calcular as probabilidades que determinados documentos estejam em determinados tópicos vimos que poucos tópicos determinam um documento, independente de usarmos 13 ou 100 tópicos no modelo. Para calcular a matrix esparsa das similaridades eu acabei tendo que preencher um array com 0 em quase todas as posições para que não desse erro.
Devemos estudar para ver se o modelo LDA é realmente a melhor abordagem neste caso ou se não temos muitos poucos documentos.

Sem preencher com 0's os tópicos que não contribuiam para cada documento o que acontecia era que cada array de probabilidade por tópico de um documento tinha um tamanho diferente, o que dava erro ao criar a matrix esparsa, que espera que todos tenham a mesma quantidade de colunas, que no caso é a quantidade de tópicos.

In [169]:
similarity_calculator = SimilarityCalculator(probabilidades_topicos_wikipedia)

### Calcula as paginas da wikipedia mais parecidas com os posts do nerds viajantes

In [170]:
wikipedia_mais_parecidos = [similarity_calculator.get_most_similar_documents(pnv) for pnv in probabilidades_topicos_nerds_viajantes]

### Uma análise de post específico (bariloche-cerro-tronador-e-cascada-los-alerces)

Post escolhido do Nerds viajantes para analisar:

- id: 1356
- indice no array: 49
- name: bariloche-cerro-tronador-e-cascada-los-alerces

In [180]:
indice_post_nv = 49
posts.iloc[49][['name', 'title']]

name       bariloche-cerro-tronador-e-cascada-los-alerces
title    Bariloche - Cerro Tronador e Cascada los Alerces
Name: 49, dtype: object

In [135]:
indice_wikipedia = 1145
print(f'Pagina da wikipedia: {pages_df.iloc[0].title}')

Pagina da wikipedia: Geografia da Argentina


In [182]:
paginas_wikipedia_mais_parecidas = wikipedia_mais_parecidos[indice_post_nv]
print(f'Wikipedia mais parecido: {pages_df.iloc[paginas_wikipedia_mais_parecidas, [1, 2]]}')

Wikipedia mais parecido:                                               title         country
1119                                    Rio Awatere   Nova Zelandia
2185                                     Rio Innoko  Estados Unidos
1109                                      Rio Arrow   Nova Zelandia
1194                                 Albion (Texas)  Estados Unidos
1258                                 Boston (Texas)  Estados Unidos
1789  Parque Nacional e Reserva Gates of the Arctic  Estados Unidos
1124                                    Rio Barrier   Nova Zelandia
894                   Lista de atrações de Auckland   Nova Zelandia
2188                                    Rio Nowitna  Estados Unidos
1138                                   Rio Turakina   Nova Zelandia


**Comentários sobre o resultado e trabalho futuro**

- Todos os posts que vem como os mais parecidos são relacionados a rios
- Os rios dos resultados são distribuídos entre os países Estados Unidos e Nova Zelândia mas curiosamente não Argentina
    * Provavelmente eu não baixei os posts de rios da Argentina

**Trabalho futuro**

- Analisar qual tópico mais aparece para este post
    * Analisar também do mais parecido na wikipedia
- Analisar palavras que mais contribuem para os tópicos em questão